In [7]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds
import tqdm
import pandas as pd
import argparse

df = pd.read_feather("cleaned_images.df").set_index("key")
train_df = df.loc[df.ds_type=="train"]
val_df = df.loc[df.ds_type=="val"]
test_df = df.loc[df.ds_type=="test"]

df.value_counts("ds_type")

ds_type
train    128689
val       16547
test      16242
dtype: int64

In [ ]:
model = tf.keras.models.load_model()

def mapping(path,label):

    image = tf.io.read_file(path)
    image = tf.io.decode_jpeg(image, channels=3)
    label = tf.one_hot(label,2)
    return image, label

def get_ds(df,bs):
    ds = tf.data.Dataset.from_tensor_slices((df.local_path.values,(df.view_direction=="Sideways").apply(int).values))

    ds = ds.map(mapping).shuffle(1000).batch(bs).prefetch(tf.data.AUTOTUNE)

    return ds
    
val_ds = get_ds(val_df)

In [ ]:
preds = model.predict(ds,verbose=1)

In [2]:
df.value_counts(["ds_type","view_direction"],normalize=True)

ds_type  view_direction
train    Forward           0.540185
         Sideways          0.252901
test     Forward           0.068313
val      Forward           0.065922
         Sideways          0.034754
test     Sideways          0.032240
train    Backward          0.003858
val      Backward          0.001796
test     Backward          0.000031
dtype: float64